## Inference

### Import packages

In [ ]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


### Select detection model

In [ ]:
#@markdown ### Enter the experiment name for detection model:
detection_model_exp_name = 'exp' #@param {type:"string"}

detection_model_path = f"/root/autodl-tmp/DT_SegNet/Detection_Model_Output/{detection_model_exp_name}/weights/best.pt"


Or use our trained detection model

In [ ]:
# download our trained best model

Path('/root/autodl-tmp/DT_SegNet/Models').mkdir(exist_ok=True, parents=True)

!wget -O /root/autodl-tmp/DT_SegNet/Models/detection.pt https://github.com/xiazeyu/DT_SegNet/releases/latest/download/detection.pt

detection_model_path= "/root/autodl-tmp/DT_SegNet/Models/detection.pt"


Then, ensure correct detection model is selected

In [ ]:
!stat {detection_model_path}

## Select segmentation model

In [ ]:
#@markdown ### Enter the experiment name for segmentation model:
segmentation_model_exp_name = 'B1' #@param {type:"string"}

segmentation_model_path = f"/root/autodl-tmp/DT_SegNet/Segmentation_Model_Output/{segmentation_model_exp_name}/best_model/model.pdparams"


Or use our trained segmentation model

In [ ]:
# download our trained best model

Path('/root/autodl-tmp/DT_SegNet/Models').mkdir(exist_ok=True, parents=True)

!wget -O /root/autodl-tmp/DT_SegNet/Models/segmentation.pdparams https://github.com/xiazeyu/DT_SegNet/releases/latest/download/segmentation.pdparams

segmentation_model_path= "/root/autodl-tmp/DT_SegNet/Models/segmentation.pdparams"


Then, ensure correct segmentation model is selected

In [ ]:
!stat {segmentation_model_path}

### Inference

In [ ]:
# read YOLO format txt label, and perform dilation
def read_labels(label_path, img, dilation = 1.5):
    data = pd.read_csv(str(label_path), sep=" ", header=None,
                    names=["class", "x_center", "y_center", "width", "height", "proability"])
    img_x, img_y = img.size
    data['x0'] = ((data['x_center'] - dilation * data['width'] / 2) * img_x).astype("int")
    data['x1'] = ((data['x_center'] + dilation * data['width'] / 2) * img_x).astype("int")
    data['y0'] = ((data['y_center'] - dilation * data['height'] / 2) * img_y).astype("int")
    data['y1'] = ((data['y_center'] + dilation * data['height'] / 2) * img_y).astype("int")
    return data

In [ ]:
%cd 1_Detection_Model
!python detect.py --project "/root/autodl-tmp/DT_SegNet/Detection_Output" --weights {detection_model_path} --img 1280 --source /root/autodl-tmp/DT_SegNet/Dataset/test --line-thickness 2 --save-txt --save-conf --save-crop --conf-thres 0.475
%cd ..

In [ ]:
#@markdown ### Enter the experiment name for detection inference:
detection_inference_exp = 'exp' #@param {type:"string"}

detection_inference_exp_path = f"/root/autodl-tmp/DT_SegNet/Detection_Output/{detection_inference_exp}"


In [ ]:
data_dir = Path('/root/autodl-tmp/DT_SegNet/Dataset/test')
label_dir = Path(detection_inference_exp_path) / 'labels'

seg_output_dir = Path('/root/autodl-tmp/DT_SegNet/Dataset/Segmentation_Input/')

In [ ]:
seg_output_dir.mkdir(parents=True, exist_ok=True)
for img_path in list(data_dir.glob('**/*.png')):
    print(f'Processing {str(img_path)}')

    # process cropped image
    img = Image.open(img_path)
    img = img.convert("L")
    # print(img.format, img.size, img.mode)
    labels = read_labels((label_dir / img_path.name).with_suffix('.txt'), img)
    with tqdm(total=len(labels)) as pbar:
        for index, r in labels.iterrows():
            box = (r.x0, r.y0, r.x1, r.y1)
            region = img.crop(box)
            # region.show()
            croped_savepath = seg_output_dir / f'{img_path.stem}_{index}{img_path.suffix}'
            # print(croped_savepath, box)
            region.save(croped_savepath)
            pbar.update(1)

In [ ]:
%cd 3_Segmentation_Model
!python predict.py --config configs/dtsegnet/segformer_b1.yml --model_path {segmentation_model_path} --image_path /root/autodl-tmp/DT_SegNet/Dataset/Segmentation_Input/ --save_dir /root/autodl-tmp/DT_SegNet/Segmentation_Output/
%cd ..

In [ ]:
data_dir = Path('/root/autodl-tmp/DT_SegNet/Dataset/test')
seg_output_dir = Path('/root/autodl-tmp/DT_SegNet/Segmentation_Output/pseudo_color_prediction')
label_dir = Path(detection_inference_exp_path) / 'labels'
output_root = Path('/root/autodl-tmp/DT_SegNet/Output')
output_root.mkdir(exist_ok=True, parents=True)

In [ ]:
for img_path in list(data_dir.glob('*.png')):
    print(f'Processing {str(img_path)}')
    img = Image.open(img_path)
    img = img.convert("L")
    labels = read_labels((label_dir / img_path.name).with_suffix('.txt'), img)
    output = np.zeros_like(img)
    with tqdm(total=len(labels)) as pbar:
        for index, r in labels.iterrows():
            croped_path = seg_output_dir / f'{img_path.stem}_{index}{img_path.suffix}'
            region = Image.open(croped_path)
            np_region = np.array(region)
            x0, x1, y0, y1 = int(r.x0), int(r.x1), int(r.y0), int(r.y1)
            for x in range(x1-x0):
                for y in range(y1-y0):
                    y_out = y+y0
                    x_out = x+x0
                    y_out = max(0, y_out)
                    x_out = max(0, x_out)
                    y_out = min(output.shape[0]-1, y_out)
                    x_out = min(output.shape[1]-1, x_out)
                    output[y_out, x_out] += np_region[y, x] # add regions
                    # output[y_out, x_out] = np_region[y, x] # replace regions
            pbar.update(1)

    output[output>=1]=1
    data = np.array(output)
    new_data = data.astype('uint8')
    print(data.shape, new_data.shape, np.unique(new_data))
    plt.imshow(new_data, interpolation='nearest', cmap='Greys')
    # white background
    plt.show()
    np.save(output_root / f'{img_path.stem}.npy', new_data)
    Image.fromarray(
        new_data*255).save(output_root / f'{img_path.stem}.png')
